In [4]:
import pandas as pd
import numpy as np

from skopt.space import Real, Integer
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

from scipy.stats import qmc  

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [5]:
df = pd.read_csv('Updated_Drone_Data_Modified.csv')

df

,Maximum Principal Elastic Strain (m/m),Total Deformation (m),Equivalent Elastic Strain (m/m),Sample Volume (mm³),Strain to Deformation Ratio,Length (mm),Breadth (mm),Depth (mm)
0,1.650000e-08,1.190000e-09,2.170000e-08,16400000,13.783160,403.1,406.3,101.0
1,1.460000e-08,1.060000e-09,1.930000e-08,16400000,13.776398,399.9,412.5,100.9
2,1.280000e-08,9.290000e-10,1.690000e-08,16400000,13.767023,403.0,412.7,99.5
3,1.100000e-08,7.960000e-10,1.450000e-08,16400000,13.772023,397.8,412.7,99.7
4,9.170000e-09,6.630000e-10,1.210000e-08,16400000,13.830928,398.0,408.9,99.1
...,...,...,...,...,...,...,...,...
65,2.920000e-09,2.120000e-10,3.860000e-09,24766722,13.770127,457.8,389.6,138.2
66,3.000000e-09,2.180000e-10,3.960000e-09,25490140,13.770127,397.0,445.2,145.6
67,3.080000e-09,2.230000e-10,4.060000e-09,18053160,13.770127,375.4,418.0,115.9
68,3.160000e-09,2.290000e-10,4.170000e-09,21779712,13.770127,382.1,413.1,137.5


In [6]:
# Scale the 'Maximum Principal Elastic Strain' column by 10^10
df['Maximum Principal Elastic Strain (m/m)'] = df['Maximum Principal Elastic Strain (m/m)'] * 1e10

# Optionally, check the updated values
print(df.head())

   Maximum Principal Elastic Strain (m/m)  Total Deformation (m)  \
0                                   165.0           1.190000e-09   
1                                   146.0           1.060000e-09   
2                                   128.0           9.290000e-10   
3                                   110.0           7.960000e-10   
4                                    91.7           6.630000e-10   

   Equivalent Elastic Strain (m/m)  Sample Volume (mm³)  \
0                     2.170000e-08             16400000   
1                     1.930000e-08             16400000   
2                     1.690000e-08             16400000   
3                     1.450000e-08             16400000   
4                     1.210000e-08             16400000   

   Strain to Deformation Ratio  Length (mm)  Breadth (mm)  Depth (mm)  
0                    13.783160        403.1         406.3       101.0  
1                    13.776398        399.9         412.5       100.9  
2                   

In [7]:
# Define the hyperparameter space
hyp_space  = [
    Integer(3, 15, name='num_layers'), # Num of layers in the network (depth)
    Integer(50, 300, name='num_units'), # Num of neurons in each hidden layer (width)
    Real(0.00001, 0.2, prior='log-uniform', name='learning_rate'), # Steps size at each iteration 
    Real(0.0, 0.7, name='dropout_rate'), # Probability of droping out a neuron
    Integer(20, 200, name='batch_size'), # Num of samples per batch
    Integer(50, 250, name='epochs') # Num of epochs (iterations over the entire dataset) during training
]

# Defining the Neural Network Model
def NN_model(num_layers, num_units, learning_rate, dropout_rate):
    inputs = Input(shape=(3,))
    x = Dense(num_units, activation='relu', kernel_regularizer='l2')(inputs)
    for _ in range(num_layers - 1):
        x = Dense(num_units, activation='relu', kernel_regularizer='l2')(x)
        x = Dropout(dropout_rate)(x)
    outputs = Dense(1, activation='linear')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model

# Function for MC Dropout predictions
def mc_dropout_predictions(model, X, num_samples=50):
    predictions = np.zeros((num_samples, X.shape[0]))
    for i in range(num_samples):
        predictions[i, :] = model(X, training=True).numpy().flatten()
    prediction_mean = predictions.mean(axis=0)
    prediction_std = predictions.std(axis=0)
    return prediction_mean, prediction_std

In [8]:
# Define the objective function to minimize
@use_named_args(hyp_space)
def objective(**params):
    num_layers = params['num_layers']
    num_units = params['num_units']
    learning_rate = params['learning_rate']
    dropout_rate = params['dropout_rate']
    batch_size = params['batch_size']
    epochs = params['epochs']

    model = NN_model(num_layers, num_units, learning_rate, dropout_rate)

    # Define K-fold cross-validation
    kfold = KFold(n_splits=3, shuffle=True, random_state=0)
    scores = []

    # Perform cross-validation
    for train_idx, val_idx in kfold.split(df[['Length (mm)', 'Breadth (mm)', 'Depth (mm)']]):
        X_train, X_val = df[['Length (mm)', 'Breadth (mm)', 'Depth (mm)']].iloc[train_idx], df[['Length (mm)', 'Breadth (mm)', 'Depth (mm)']].iloc[val_idx]
        y_train, y_val = df[['Maximum Principal Elastic Strain (m/m)']].iloc[train_idx], df[['Maximum Principal Elastic Strain (m/m)']].iloc[val_idx]

        # Standardize the features
        scaler_x = StandardScaler().fit(X_train)
        X_train_scaled = scaler_x.transform(X_train)
        X_val_scaled = scaler_x.transform(X_val)

        # Train the model
        # early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        model.fit(X_train_scaled, y_train, epochs=epochs, batch_size=batch_size, verbose=0, validation_data=(X_val_scaled, y_val))

        # Evaluate the model
        score = model.evaluate(X_val_scaled, y_val, verbose=0)
        scores.append(score)
    
    return np.mean(scores)

# Perform Bayesian optimization
result = gp_minimize(objective, hyp_space, n_calls=30, random_state=0, acq_func='EI')

# Output best hyperparameters from BayesOpt
print("Best hyperparameters:")
print("num_layers:", result.x[0])
print("num_units:", result.x[1])
print("learning_rate:", result.x[2])
print("dropout_rate:", result.x[3])
print("batch_size:", result.x[4])
print("epochs:", result.x[5])

# Train the model with the best hyperparameters
best_model = NN_model(
    num_layers=result.x[0],
    num_units=result.x[1],
    learning_rate=result.x[2],
    dropout_rate=result.x[3],
)

Best hyperparameters:
num_layers: 4
num_units: 296
learning_rate: 0.038098440670222465
dropout_rate: 0.07336707667550668
batch_size: 170
epochs: 67


In [9]:
# Standardize the initial dataset
scaler_x = StandardScaler().fit(df[['Length (mm)', 'Breadth (mm)', 'Depth (mm)']])
X_scaled = scaler_x.transform(df[['Length (mm)', 'Breadth (mm)', 'Depth (mm)']])

# Train the best model on the initial dataset
best_model.fit(X_scaled, df[['Maximum Principal Elastic Strain (m/m)']], epochs=result.x[5], batch_size=result.x[4], verbose=1)

# # Produce Meshgrid of results with C.I. 
# x1_range = np.linspace(360, 470, 100)
# x2_range  = np.linspace(360, 470, 100)

# x1_grid, x2_grid = np.meshgrid(x1_range, x2_range)
# x_grid = np.c_[x1_grid.ravel(), x2_grid.ravel()]

# x_grid = scaler_x.transform(x_grid)

# # Perform MC Dropout predictions for entire meshgrid
# pred_mean, pred_std = mc_dropout_predictions(best_model, x_grid)

# # Define range for inputs
# bounds = np.array([[360, 470], [360, 470]])

# # Identify top 10 points with highest uncertainty
# num_new_points = 10

# sampler = qmc.LatinHypercube(d=2)  # LHS in a 2D space
# lhs_sample = sampler.random(n=1000)
# lhs_points = qmc.scale(lhs_sample, bounds[:, 0], bounds[:, 1])

# lhs_points_standardized = scaler_x.transform(lhs_points)

# lhs_mean, lhs_std = mc_dropout_predictions(best_model, lhs_points_standardized)

# percentile_threshold = 90 
# threshold_value = np.percentile(lhs_mean, percentile_threshold)

# # Filter points above the threshold
# points_above_threshold = lhs_points[lhs_mean >= threshold_value]

# num_new_points = 10
# if len(points_above_threshold) > num_new_points:
#     selected_indices = np.random.choice(len(points_above_threshold), num_new_points, replace=False)
#     selected_points = points_above_threshold[selected_indices]
# else:
#     selected_points = points_above_threshold

# new_points_df = pd.DataFrame(selected_points, columns=['x','y'])

# # Ploting mean predictions and confidence intervals
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# # Reshaping predictions back to grid shape
# pred_mean_grid = pred_mean.reshape(x1_grid.shape)
# pred_upper_grid = (pred_mean + (1.96 * pred_std)).reshape(x1_grid.shape)
# pred_lower_grid = (pred_mean - (1.96 * pred_std)).reshape(x1_grid.shape)

# mean_surface = ax.plot_surface(x1_grid, x2_grid, pred_mean_grid, color='blue', alpha=0.5, label='Mean Prediction')
# upper_surface = ax.plot_surface(x1_grid, x2_grid, pred_upper_grid, color='red', alpha=0.3, label='Upper Bound (95% CI)')
# lower_surface = ax.plot_surface(x1_grid, x2_grid, pred_lower_grid, color='green', alpha=0.3, label='Lower Bound (95% CI)')

# # Plot next points for sampling
# ax.scatter(selected_points[:, 0], selected_points[:, 1], color='black', marker='o', label='Next Sampling Points')

# # Plot Labels
# ax.set_title('Neural Network Predictions with Confidence Intervals')
# ax.set_xlabel('Length (mm)')
# ax.set_ylabel('Breadth (mm)')
# ax.set_zlabel('Equivalent Elastic Strain (m/m)')

# # Legend
# legend_elements = [mean_surface, upper_surface, lower_surface]
# fig.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(0.1, 0.9))

# # Show the plot
# plt.show()

Epoch 1/67
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 3538.9084
Epoch 2/67
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 34540.0703
Epoch 3/67
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 3367.7156
Epoch 4/67
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 3533.7578
Epoch 5/67
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 3529.1094
Epoch 6/67
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 3517.2681
Epoch 7/67
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 3497.8708
Epoch 8/67
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 3463.2358
Epoch 9/67
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 3414.7422
Epoch 10/67
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 3373.0408
Epoch 11/67
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 3256.9773
Epoch 12/67
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 3056.9055
Epoch 13/67
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 2732.3027
Epoch 14/67
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 2461.9292
Epoch 15/67
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 

In [12]:
from sklearn.model_selection import train_test_split

train_X = df[['Length (mm)', 'Breadth (mm)', 'Depth (mm)']].to_numpy()
train_y = df['Maximum Principal Elastic Strain (m/m)'].to_numpy()

# Standardize the initial dataset
scaler_x = StandardScaler().fit(train_X)
X_scaled = scaler_x.transform(train_X)

In [13]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred = best_model.predict(X_scaled)

# # Calculate regression metrics
mse = mean_squared_error(train_y, y_pred)
mae = mean_absolute_error(train_y, y_pred)
r2 = r2_score(train_y, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R² Score: {r2}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Mean Squared Error (MSE): 651.1357771449027
Mean Absolute Error (MAE): 15.36483154296875
R² Score: 0.5019848664673041
